<a href="https://colab.research.google.com/github/ithabibi/crawler_google_trends/blob/main/monthlyqueryMCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google trends **scraper**

## Lib import
install and import necessary library

In [ ]:
!pip install pytrends
# import matplotlib
import pandas as pd
from pytrends.request import TrendReq
import plotly.express as px
import locale
import sys  # for utf8
import datetime
from dateutil.relativedelta import relativedelta
import time

import os.path
from pathlib import Path

# Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%pwd
%cd /content/gdrive/MyDrive/
%pwd

## Run crawler
run scraper for retrieval key-phrase searched in google trends

In [ ]:
%cd /content/gdrive/MyDrive # set default directory
%pwd
!mkdir scrap 
trend1 = "همراه اول" #@param {type:"string"}
trend2 = "ایرانسل" #@param {type:"string"}

start_date = datetime.date(2017, 1, 1)#@param {type:"string"}
end_date = datetime.date(2019, 3, 1)#@param {type:"string"}

appended_data = pd.DataFrame()
# set utf8 for terminal
#sys.stdout.reconfigure(encoding='utf-8')
print("به عنوان کدینگ پیش فرض فعال شد", sys.getdefaultencoding())
# display all rows from data frame using Pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# add my parameters
#trend = TrendReq(hl='fa', tz=330,)
# keyword list
kw_list = [trend1, trend2]


###########  while  ######################################################################################
delta = datetime.timedelta(days=1)

while start_date <= end_date:
  time_slice = str(start_date) + " " + str(start_date + relativedelta(months=1))
  if os.path.isfile('/content/gdrive/MyDrive/scrap/' + str(start_date)+".csv"): # check exist file
    print(str(start_date)+".csv is exist")
    start_date = start_date + relativedelta(months=+1)
    continue
  trend = TrendReq(hl='fa', tz=330,)
  trend.build_payload(kw_list, cat=0, timeframe=time_slice, geo='', gprop='')
  rq = trend.related_queries()
  rq.values()

  #Retrieval rising trend1
  df = pd.DataFrame(rq.get(trend1).get('rising'))
  df = df.assign(date=start_date, keyword=trend1, get_type='rising')
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)

  df = pd.DataFrame(rq.get(trend1).get('top'))
  df = df.assign(date=start_date, keyword=trend1 , get_type='top' )
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)

  #Retrieval rising trend2
  df = pd.DataFrame(rq.get(trend2).get('rising'))
  df = df.assign(date=start_date, keyword=trend2, get_type='rising')
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)

  df = pd.DataFrame(rq.get(trend2).get('top'))
  df = df.assign(date=start_date, keyword=trend2, get_type='top')
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)
  
  appended_data.to_csv('/content/gdrive/MyDrive/scrap/' + str(start_date) + '.csv')
  print(str(start_date) + " OK")
  appended_data.drop(df.index, inplace=True)
  start_date = start_date + relativedelta(months=+1)

  # politeness Scraping
  del trend
  time.sleep(5) 
  
appended_data = appended_data.reset_index()
print(appended_data.head(10))
#print(str(pd.read_csv(str(start_date + relativedelta(months=-1)) + ".csv")))

#appended_data.to_csv(time_slice + '.csv')

## Merg CSV **file**
In order to download files more easily, you can merge them together

In [ ]:
import glob

# list all csv files only
csv_files = glob.glob('scrap/' +'*.{}'.format('csv'))
print(str(csv_files))

df_csv_append = pd.DataFrame()
 
# append the CSV files
for file in csv_files:
    df = pd.read_csv(file)
    df_csv_append = df_csv_append.append(df, ignore_index=True)
df_csv_append.to_csv('scrap/merged_data.csv')
df_csv_append.head(10)

## Download folder From google colab
You can also upload the collected files to your Google Drive. Or download the files for later use.

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive/
!zip -r colabcontent.zip  /content/scrap/*.csv -j # The j option doesn't store directory information.

from google.colab import files
files.download('colabcontent.zip')

## Upload file from google drive to google colab
You can upload files that you have already uploaded to your Google Drive. Re-upload to Google Collab.

In [ ]:
!cp /content/gdrive/MyDrive/colabcontent.zip /content/scrap/
%cd /content/scrap/
!unzip colabcontent.zip